# Multinomial Classification



### Imports

In [2]:
import numpy as np # numpy를 np로써 불러온다
import torch # torch를 import
import torch.nn as nn # torch.nn을 nn으로써 불러온다
import torch.nn.functional as func # torch.nn.functional를 func로써 불러온다
import torch.optim as opt # torch.optim를 opt로써 불러온다

from google.colab import drive
drive.mount('/content/drive') # google로 mount하는 과정

# Seed 고정
torch.manual_seed(1)

Mounted at /content/drive


### 1 Data loading
| $x_1$ | $x_2$ | $x_3$ | $x_4$ | $y$ |
|---|---|---|---|---|
| 1 | 2 | 1 | 1 | 2 |
| 2 | 1 | 3 | 2 | 2 |
| 3 | 1 | 3 | 4 | 2 |
| 4 | 1 | 5 | 5 | 1 |
| 1 | 7 | 5 | 5 | 1 |
| 1 | 2 | 5 | 6 | 1 |
| 1 | 6 | 6 | 6 | 0 |
| 1 | 7 | 7 | 7 | 0 |

In [3]:
# Data 수동으로 입력하기
x_train = torch.FloatTensor([[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5],
               [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

print(x_train) # x_train 출력
print(x_train.shape) # x_train 모양 출력
print(y_train) # y_train 출력
print(y_train.shape) # y_train 모양 출력

# Model 초기화 (입력 dim, 출력 dim)
model = nn.Linear(4, 3) # 4입력 -> 3출력

tensor([[1., 2., 1., 1.],
        [2., 1., 3., 2.],
        [3., 1., 3., 4.],
        [4., 1., 5., 5.],
        [1., 7., 5., 5.],
        [1., 2., 5., 6.],
        [1., 6., 6., 6.],
        [1., 7., 7., 7.]])
torch.Size([8, 4])
tensor([2, 2, 2, 1, 1, 1, 0, 0])
torch.Size([8])


### 2 Hypothesis, Cost and Optimization
* 다음 수식이 맞다고 가정한다
$$ y_{hypo} = H(x) = Softmax( Linear(x) ) $$
$$ Linear(x) = W \cdot x + b $$
$$ Softmax(X) = [y_1, ..., y_C], \;\; y_i = \frac{e^{x_i}}{\sum_{j=1}^C e^{x_j}} $$

* Cost function으로 Cross Entropy를 이용한다
$$ - \sum_{i=1}^{C} y_{train, i}log(y_{hypo, i})$$

* Pytorch의 cross entropy loss function에는 softmax가 이미 포함되어 있음에 주의!
  + **그렇기 때문에, model을 그냥 linear model을 사용한다!**
  + y_train 입력 형태는 class의 index이다 (onehot encoding을 해줄 필요 없다)

In [ ]:
# Optimizer 설정 (learning rate = 0.1로 설정)
optimizer = opt.SGD(model.parameters(), lr=0.1)

# 1000번 반복한다
for epoch in range(1000):

  # Cost 계산 / mse_loss(가정에의한값, 참값)
  y_hypo = model(x_train)
  cost = func.cross_entropy(y_hypo, y_train)

  # cost를 이용해 model update
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 10번 마다 중간결과 출력
  if epoch % 100 == 99:
    correct_prediction = y_hypo.argmax(dim=1) == y_train
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, 1000, cost.item(), accuracy * 100,
    ))

Epoch   99/1000 Cost: 0.695567 Accuracy 75.00%
Epoch  199/1000 Cost: 0.614224 Accuracy 75.00%
Epoch  299/1000 Cost: 0.556557 Accuracy 75.00%
Epoch  399/1000 Cost: 0.505776 Accuracy 87.50%
Epoch  499/1000 Cost: 0.457821 Accuracy 87.50%
Epoch  599/1000 Cost: 0.411135 Accuracy 87.50%
Epoch  699/1000 Cost: 0.364998 Accuracy 87.50%
Epoch  799/1000 Cost: 0.319251 Accuracy 87.50%
Epoch  899/1000 Cost: 0.275260 Accuracy 87.50%
Epoch  999/1000 Cost: 0.244028 Accuracy 100.00%


### 3 Assignment
* Data 파일을 이용한 Multinomial Classification
  - data_multinomial_classification.csv을 이용해 학습한다
    + 16개의 input, 1개의 output
    
$$ [x_1, ... x_{16}, y] $$
  - Accuracy 95% 이상을 달성한다

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as opt

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Numpy를 이용해 csv file을 ndarray로 불러온다
dataset = np.loadtxt(
    '/content/drive/MyDrive/data_multinomial_classification.csv',  # 구글드라이브에 있는 파일 가져온다
    delimiter=',', # 파일의 데이터를 ,(콤마) 형태로 구분한다
    dtype=np.float32) # float 형태로 가져온다

# random
np.random.shuffle(dataset) # 순서를 random으로 섞기

#데이터 셋 나누기
x_train = torch.FloatTensor(dataset[:,:-1]) # 전체중에 앞에 16개는 x / 맨 마지막 한개 빼고 다 x_train
y_train = torch.LongTensor(dataset[:,-1]) # 전체중에 뒤에 1개는 y / 묶어서 가져오면 안되기 때문에 [-1]을 해준다

# Model 초기화 (입력 dim, 출력 dim)
model = nn.Linear(16, 7) # 입력이 16, 출력은 7개

# Optimizer 설정
optimizer = opt.SGD(model.parameters(), lr=0.1) # SGD 함수를 이용하여 model의 parameters값 최적화. learning rate는 0.1

In [6]:
# 반복
for epoch in range(1000): # 1000번 반복

  # Cost 계산 / mse_loss(가정에의한값, 참값)
  y_hypo = model(x_train)
  cost = func.cross_entropy(y_hypo, y_train)


  optimizer.zero_grad() # gradient 초기화
  cost.backward() # w와 b 기울기 업데이트
  optimizer.step() # 모델을 수정하는 과정

  # 10번 마다 중간결과 출력
  if epoch % 100 == 99: # 나머지가 99일때 마다 출력한다
    correct_prediction = y_hypo.argmax(dim=1) == y_train # argmax함수 = 백터중에 max값을 추출한다
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('Epoch {}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, 1000, cost.item(), accuracy * 100, # epoch, cost, accuracy 출력
    ))

Epoch 99/1000 Cost: 0.469938 Accuracy 89.11%
Epoch 199/1000 Cost: 0.317336 Accuracy 93.07%
Epoch 299/1000 Cost: 0.245835 Accuracy 95.05%
Epoch 399/1000 Cost: 0.202133 Accuracy 96.04%
Epoch 499/1000 Cost: 0.172118 Accuracy 97.03%
Epoch 599/1000 Cost: 0.150037 Accuracy 100.00%
Epoch 699/1000 Cost: 0.133041 Accuracy 100.00%
Epoch 799/1000 Cost: 0.119530 Accuracy 100.00%
Epoch 899/1000 Cost: 0.108523 Accuracy 100.00%
Epoch 999/1000 Cost: 0.099384 Accuracy 100.00%
